In [8]:

import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import random

In [2]:
procedure = ['Healthy', 'COLPOs','LEEPs']
results_folders=[]

for folder_name in procedure:
    main_path = 'R:\\Cervical_Project\\Data\\' + folder_name
    os.chdir(main_path)
    files = os.listdir()
    for file in files:
        new_path = os.path.join(main_path, file)
        print(new_path)
        if os.path.isdir(new_path) and 'results_final' in os.listdir(new_path):
            paired_path = os.path.join(new_path, 'results_final')
            results_folders.append(paired_path)

R:\Cervical_Project\Data\Healthy\20191003_Hysterectomy
R:\Cervical_Project\Data\Healthy\20191031_Hysterectomy
R:\Cervical_Project\Data\Healthy\20191107_Hysterectomy
R:\Cervical_Project\Data\Healthy\20191121_Hysterectomy
R:\Cervical_Project\Data\Healthy\20200220_Hyterectomy
R:\Cervical_Project\Data\Healthy\20200305_Hysterectomy
R:\Cervical_Project\Data\Healthy\20200928_Hysterectomy
R:\Cervical_Project\Data\Healthy\20201008_Hysterectomy
R:\Cervical_Project\Data\Healthy\20201119_Hysterectomy
R:\Cervical_Project\Data\Healthy\20201127_Hysterectomy
R:\Cervical_Project\Data\Healthy\20201214_Hysterectomy
R:\Cervical_Project\Data\Healthy\20210218_Hysterectomy
R:\Cervical_Project\Data\Healthy\20210415_Hysterectomy
R:\Cervical_Project\Data\Healthy\20210423_Hysterectomy
R:\Cervical_Project\Data\Healthy\20210526_Hysterectomy
R:\Cervical_Project\Data\Healthy\20210715_Hysterectomy
R:\Cervical_Project\Data\Healthy\20211104_Hysterectomy
R:\Cervical_Project\Data\Healthy\20220331_Hysterectomy
R:\Cervical

In [3]:
results_folders

['R:\\Cervical_Project\\Data\\Healthy\\20191003_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20191031_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20191107_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20191121_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20200220_Hyterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20200305_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20200928_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20201008_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20201119_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20201127_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20201214_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20210218_Hysterectomy\\results_final',
 'R:\\Cervical_Project\\Data\\Healthy\\20210415_Hysterectomy\\res

In [4]:
def normalize(image, p_min=2, p_max=99.9, dtype='float32'):
    '''
    Normalizes the image intensity so that the `p_min`-th and the `p_max`-th
    percentiles are converted to 0 and 1 respectively.
    References
    ----------
    Content-Aware Image Restoration: Pushing the Limits of Fluorescence
    Microscopy
    https://doi.org/10.1038/s41592-018-0216-7
    '''
    low, high = np.percentile(image, (p_min, p_max))
    return (image - low) / (high - low + 1e-6)

def patch_image(image):
    '''Image should be 512x512'''
    return image.reshape(2, 256, 2, 256).swapaxes(1, 2).reshape(4, 256, 256)

def stitch_image(image):
    '''Image should be 4x256x256'''
    return np.reshape(image,[2, -1, 256, 256]).swapaxes(1,2).reshape(512, 512)

def create_patches(input,start,end):
    start = int(start)
    end = int(end)
    indices = list(range(start,end+1))
    total_patches = []
    for i in indices:
        image = input[i]
        image = np.transpose(image)
        image = normalize(image,2,99.9)
        patches = patch_image(image)
        total_patches.append(patches)
    #total_patches = np.swapaxes(total_patches,0,1)\
    total_patches = np.reshape(total_patches,(-1,256,256))
    return list(total_patches)

def Patchify(f,roi_name):
    try: 
        start_stack = f['stack_begin'][0]-1
        end_stack = f['stack_end'][0]-1
        FAD= f['FAD_frames_512_shifted']
        NADH= f['NADH_frames_512_shifted']
        stack_num = end_stack - start_stack + 1
        assert stack_num > 0
        low_SNR_NADH = create_patches(NADH[3],start_stack,end_stack)
        high_SNR_NADH = create_patches(np.mean(NADH, axis=0),start_stack,end_stack)
        low_SNR_FAD = create_patches(FAD[3],start_stack,end_stack)
        high_SNR_FAD = create_patches(np.mean(FAD, axis=0),start_stack,end_stack)
        print('Start and end found: ' + roi_name + ": images=" + str(stack_num))
        return low_SNR_NADH , high_SNR_NADH, low_SNR_FAD, high_SNR_FAD, stack_num
    except:
        print('Data was not processed: ' + roi_name)
        return [],[],[],[],[]

In [5]:
# Seed here is constant as we only ever want to generate ONE identical Training and Test Set
# TODO (nvora01): Design a better way to specify how many ROIs we have
ROIs = 152
testing_split = round(ROIs*1/3)
random.seed(0) 
Training_idx = sorted(random.sample(range(0,ROIs), ROIs-testing_split))
Testing_idx = sorted([x for x in list(range(0,ROIs)) if x not in Training_idx])

[0, 1, 2, 4, 5, 7, 8, 10, 11, 12, 13, 15, 16, 18, 23, 24, 25, 26, 28, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 48, 49, 51, 52, 55, 56, 57, 61, 62, 63, 64, 65, 66, 68, 69, 70, 72, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 89, 90, 91, 92, 93, 98, 100, 103, 104, 105, 106, 107, 110, 112, 113, 116, 117, 118, 120, 121, 122, 124, 125, 127, 129, 130, 131, 132, 134, 136, 138, 139, 140, 141, 144, 145, 146, 148, 149, 150, 151]
[3, 6, 9, 14, 17, 19, 20, 21, 22, 27, 29, 32, 34, 44, 45, 46, 47, 50, 53, 54, 58, 59, 60, 67, 71, 73, 76, 86, 88, 94, 95, 96, 97, 99, 101, 102, 108, 109, 111, 114, 115, 119, 123, 126, 128, 133, 135, 137, 142, 143, 147]


In [ ]:
size=512
X_N_train = []
X_F_train = []
Y_N_train = []
Y_F_train = []

X_N_test = []
X_F_test = []
Y_N_test = []
Y_F_test = []

start_list_train = []
end_list_train = []
key_names_train = []

start_list_test = []
end_list_test = []
key_names_test = []

ROI_counter_train = 0
ROI_counter_test = 0
stack_count_train = 0
stack_count_test = 0
run_count = 0
for result_folder in results_folders:
    roi_mats = []
    os.chdir(result_folder)
    experiment_files = os.listdir()
    for experiment_file in experiment_files:
        if experiment_file.endswith('FINAL.mat'):
            roi_mats.append(experiment_file)
    for roi in roi_mats:
        if run_count in Training_idx:
            roi_name = roi[0:-4]
            f= h5py.File(roi)
            low_SNR_NADH , high_SNR_NADH, low_SNR_FAD, high_SNR_FAD, stack_num = Patchify(f,roi_name)
            if len(low_SNR_NADH) > 0:
                key_names_train.append(roi_name)
                X_N_train.append(low_SNR_NADH)
                Y_N_train.append(high_SNR_NADH)
                X_F_train.append(low_SNR_FAD)
                Y_F_train.append(high_SNR_FAD)
                ROI_counter_train += ROI_counter_train
                start_list_train.append(int(stack_count_train*4))
                stack_count_train += stack_num 
                end_list_train.append(int(stack_count_train*4-1))
                run_count += 1
        else:
            roi_name = roi[0:-4]
            f= h5py.File(roi)
            low_SNR_NADH , high_SNR_NADH, low_SNR_FAD, high_SNR_FAD, stack_num = Patchify(f,roi_name)
            if len(low_SNR_NADH) > 0:
                key_names_test.append(roi_name)
                X_N_test.append(low_SNR_NADH)
                Y_N_test.append(high_SNR_NADH)
                X_F_test.append(low_SNR_FAD)
                Y_F_test.append(high_SNR_FAD)
                ROI_counter_test += ROI_counter_test
                start_list_test.append(int(stack_count_test*4))
                stack_count_test += stack_num 
                end_list_test.append(int(stack_count_test*4-1))
                run_count += 1
print('done')

In [42]:
# Training Set Saving
ROI_keys = [key[:(key.find('ROI')+4)] for key in key_names_train]
stack_begin = [item for item in start_list_train]
stack_end = [item for item in end_list_train]
X_n = [item for sublist in X_N_train for item in sublist]
Y_n = [item for sublist in Y_N_train for item in sublist]
X_f = [item for sublist in X_F_train for item in sublist]
Y_f = [item for sublist in Y_F_train for item in sublist]

In [49]:
np.savez('R:\\Cervical_Project\\Data\\NV_928_NADH_Training', X=X_n, Y=Y_n, SB=stack_begin, SE=stack_end, ROI_keys=ROI_keys)
np.savez('R:\\Cervical_Project\\Data\\NV_928_FAD_Training', X=X_f, Y=Y_f, SB=stack_begin, SE=stack_end, ROI_keys=ROI_keys)

In [50]:
# Testing Set Saving
ROI_keys = [key[:(key.find('ROI')+4)] for key in key_names_test]
stack_begin = [item for item in start_list_test]
stack_end = [item for item in end_list_test]
X_n = [item for sublist in X_N_test for item in sublist]
Y_n = [item for sublist in Y_N_test for item in sublist]
X_f = [item for sublist in X_F_test for item in sublist]
Y_f = [item for sublist in Y_F_test for item in sublist]

In [51]:
np.savez('R:\\Cervical_Project\\Data\\NV_928_NADH_Testing', X=X_n, Y=Y_n, SB=stack_begin, SE=stack_end, ROI_keys=ROI_keys)
np.savez('R:\\Cervical_Project\\Data\\NV_928_FAD_Testing', X=X_f, Y=Y_f, SB=stack_begin, SE=stack_end, ROI_keys=ROI_keys)

In [54]:
# Test set ROI names:
for i in ROI_keys:
    print(i)

20191121_Hysterectomy_ROI3
20200220_Hyterectomy_ROI1
20200928_Hysterectomy_ROI2
20201119_Hysterectomy_ROI1
20201127_Hysterectomy_ROI2
20201127_Hysterectomy_ROI4
20201214_Hysterectomy_ROI1
20201214_Hysterectomy_ROI2
20201214_Hysterectomy_ROI3
20210415_Hysterectomy_ROI1
20210415_Hysterectomy_ROI3
20210526_Hysterectomy_ROI1
20210526_Hysterectomy_ROI3
20201211_COLPO_ROI1
20201211_COLPO_ROI2
20201211_COLPO_ROI3
20201221_COLPO_ROI1
20201221_COLPO_ROI5
20210108_COLPO_ROI4
20210312_COLPO_ROI1
20210402_COLPO_ROI2
20210402_COLPO_ROI3
20210409_COLPO_ROI1
20210525_COLPO_ROI1
20210816_COLPO_ROI1
20210921_COLPO_ROI1
20211008_COLPO_ROI1
20200901_LEEP_ROI3
20200902_LEEP_ROI2
20200909_LEEP_ROI4
20210126_LEEP_ROI1
20210126_LEEP_ROI2
20210330_LEEP_ROI1
20210519_LEEP_ROI1
20210604_LEEP_ROI2
20210604_LEEP_ROI3
20210810_LEEP_ROI1
20210810_LEEP_ROI2
20210910_LEEP_ROI1
20210929_LEEP_ROI1
20210929_LEEP_ROI2
20211022_LEEP_ROI2
20211108_LEEP_ROI2
20211116_LEEP_ROI3
20211217_LEEP_ROI1
20211228_LEEP_ROI1
20211228_